# Feature extraction in DREAMS
some critical features

200 Hz

broadband [0,40]

In [3]:
#external libraries
import os
import dotenv
import pandas as pd
import numpy as np
from scipy import signal as sg
import pickle as pkl
import time
import matplotlib.pyplot as plt

#project library
from spinco import *

print(os.getcwd())
#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']

C:\Users\GIB\Documents\GitHub\SpinCo\SpinCo


KeyError: 'DATAPATH'

In [2]:
#define features path
dreamspath=datapath+"/DREAMS"
featurespath=dreamspath+"/features"
        
windowDurations=[0.5,1,1.5,2]

## Load data

In [3]:
signals, annotations, signalsMetadata = loadDREAMSSpindles(dreamspath)

SubjectId: 0001--------------
resampling from 100 to 200
SubjectId: 0003--------------
resampling from 50 to 200
SubjectId: 0006--------------
duration discrepancy, removing last 0.7 seconds


In [4]:
signalsMetadata.head(5)

,filename,channel,subjectId,duration,samplerate
0,excerpt1.txt,C3-A1,0001,1800,200
1,excerpt2.txt,CZ-A1,0002,1800,200
2,excerpt3.txt,C3-A1,0003,1800,200
3,excerpt4.txt,CZ-A1,0004,1800,200
4,excerpt5.txt,CZ-A1,0005,1800,200


## Preprocess

broadband in 0-40Hz, previous experiments (up tp 50Hz, are now in folders as .../features_old/...)

In [7]:
def preprocessVector(vector,samplerate):
    #1. Lowpass 40Hz
    vector=filterBand(vector,[0,40],samplerate,filterOrder=4)
    #2. Z-score
    m=np.mean(vector)
    s=np.std(vector)
    vector=(vector-m)/s
    return vector

In [8]:
for subject,signal in signals.items():
    signals[subject]=preprocessVector(signal,samplerate)

## Extract features

### folder estructure

In [9]:
fspath=featurespath+'/'+str(samplerate)+'fs/'
if not os.path.isdir(fspath):
    os.mkdir(fspath)
for window in windowDurations:
    windowPath=fspath+str(window)+'win'
    if not os.path.isdir(windowPath):
        os.mkdir(windowPath)
    for subject in signalsMetadata.subjectId:
        subjectPath=windowPath+'/'+subject
        if not os.path.isdir(subjectPath):
            os.mkdir(subjectPath)

### band definition

In [10]:
bands={
    'delta1':[0.1,2],
    'delta2':[2,4],
    'theta':[4,8],
    'alpha':[8,13],
    'sigma':[11,16],
    'beta1':[13,19],
    'beta2':[19,30]
    }

### computation

In [11]:
for window in windowDurations:
    windowPath=featurespath+'/'+str(samplerate)+'fs/'+str(window)
    for ind, row in signalsMetadata.iterrows():
        subject=row.subjectId
        subjectPath=windowPath+'/'+subject
        signal=signals[subject]
        #need to define the time vector for each signal:
        timepoints=np.arange(len(signal))/samplerate
        
        #6. Hjort Activity
        characteristic='hjortActivity'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=hjortActivity(signal,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=hjortActivity(filtered,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #7. Hjort Mobility
        characteristic='hjortMobility'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=hjortMobility(signal,timepoints,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=hjortMobility(filtered,timepoints,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #8. Hjort Complexity
        characteristic='hjortComplexity'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=hjortComplexity(signal,timepoints,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=hjortComplexity(filtered,timepoints,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #9. PetrosianFractalDimension
        characteristic='petrosian'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=petrosianFractalDimension(signal,timepoints,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=petrosianFractalDimension(filtered,timepoints,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        
        #12. sigma index
        characteristic='sigmaIndex'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=sigmaindex(signal,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
            

saving feature: D:/SpinCo/DREAMS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_broadband.fd
saving feature: D:/SpinCo/DREAMS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_delta1.fd
saving feature: D:/SpinCo/DREAMS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_delta2.fd
saving feature: D:/SpinCo/DREAMS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_theta.fd
saving feature: D:/SpinCo/DREAMS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_alpha.fd
saving feature: D:/SpinCo/DREAMS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_sigma.fd
saving feature: D:/SpinCo/DREAMS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_beta1.fd
saving feature: D:/SpinCo/DREAMS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_beta2.fd
saving feature: D:/SpinCo/DREAMS/features/200fs/0.5win/0001/0.5_0001_hjortMobility_broadband.fd
saving feature: D:/SpinCo/DREAMS/features/200fs/0.5win/0001/0.5_0001_hjortMobility_delta1.fd
saving feature: D:/SpinCo/DREAMS/features/200fs/0.5win/0001/0.5_0001_